# Nutree Overview

Nutree organizes arbitrary object instances in an unobtrusive way. <br>
That means, we can add existing objects without having to derive from a common 
base class or implement a specific protocol.

In [17]:
from nutree import Tree

tree = Tree("Hello")
tree.add("N").add("u").up(2).add("T").add("r").up().add("ee")
tree.print()

Tree<'Hello'>
├── 'N'
│   ╰── 'u'
╰── 'T'
    ├── 'r'
    ╰── 'ee'


Strings can be added to a tree, but in a real-world scenario we want need to 
handle ordinary objects:

# Setup some sample classes and objects
Let's define a simple class hierarchy

In [18]:
import uuid


class Department:
    def __init__(self, name: str):
        self.guid = uuid.uuid4()
        self.name = name

    def __str__(self):
        return f"Department<{self.name}>"


class Person:
    def __init__(self, name: str, age: int):
        self.guid = uuid.uuid4()
        self.name = name
        self.age = age

    def __str__(self):
        return f"Person<{self.name} ({self.age})>"

and create some instances

In [19]:
development_dep = Department("Development")
test__dep = Department("Test")
marketing_dep = Department("Marketing")

alice = Person("Alice", 25)
bob = Person("Bob", 35)
claire = Person("Claire", 45)
dave = Person("Dave", 55)

We have a bunch of instances now.

Let's organize these objects in a hierarchical structure using [nutree](https://nutree.readthedocs.io/):

In [20]:
from nutree import Tree

tree = Tree("Organization")

dev_node = tree.add(development_dep)
test_node = dev_node.add(test__dep)
mkt_node = tree.add(marketing_dep)

tree.add(alice)
dev_node.add(bob)
test_node.add(claire)
mkt_node.add(dave)

tree.print()

Tree<'Organization'>
├── <__main__.Department object at 0x105869d30>
│   ├── <__main__.Department object at 0x10586b200>
│   │   ╰── <__main__.Person object at 0x10586b7a0>
│   ╰── <__main__.Person object at 0x10586b680>
├── <__main__.Department object at 0x1058689e0>
│   ╰── <__main__.Person object at 0x105869f70>
╰── <__main__.Person object at 0x10586b320>


Tree nodes store a reference to the object in the `node.data` attribute.

The nodes are formatted by the object's  `__repr__` implementation by default. <br>
We can overide this by passing an [f-string](https://docs.python.org/3/tutorial/inputoutput.html#formatted-string-literals) as `repr` argument:

In [21]:
tree.print(repr="{node.data}")

Tree<'Organization'>
├── Department<Development>
│   ├── Department<Test>
│   │   ╰── Person<Claire (45)>
│   ╰── Person<Bob (35)>
├── Department<Marketing>
│   ╰── Person<Dave (55)>
╰── Person<Alice (25)>


# Access Nodes
We can use the index syntax to get the node object for a given data object:

In [22]:
tree[alice]

Node<'Person<Alice (25)>', data_id=274230066>

In [23]:
assert tree[alice].data is alice, "nodes store objects in data attribute"
alice_guid = alice.guid

# Add 

# Iteration and Searching

There are multiple methods to iterate the tree.

In [24]:
res = []
for node in tree:
    res.append(node.data.name)
print(res)

['Development', 'Test', 'Claire', 'Bob', 'Marketing', 'Dave', 'Alice']


# Mutation

# Data IDs and Clones

In [25]:
tree.print(repr="{node}", title=False)

Node<'Department<Development>', data_id=274229715>
├── Node<'Department<Test>', data_id=274230048>
│   ╰── Node<'Person<Claire (45)>', data_id=274230138>
╰── Node<'Person<Bob (35)>', data_id=274230120>
Node<'Department<Marketing>', data_id=274229406>
╰── Node<'Person<Dave (55)>', data_id=274229751>
Node<'Person<Alice (25)>', data_id=274230066>


# Serialization

In [26]:
tree.to_dict_list()

[{'data': 'Department<Development>',
  'children': [{'data': 'Department<Test>',
    'children': [{'data': 'Person<Claire (45)>'}]},
   {'data': 'Person<Bob (35)>'}]},
 {'data': 'Department<Marketing>',
  'children': [{'data': 'Person<Dave (55)>'}]},
 {'data': 'Person<Alice (25)>'}]

In [27]:
list(tree.to_list_iter())

[(0, {}), (1, {}), (2, {}), (1, {}), (0, {}), (5, {}), (0, {})]

In [28]:
t = Tree._from_list([(0, "A"), (0, "B"), (1, "C"), (0, "D"), (3, "E")])
print(t.format())

Tree<'4387670112'>
├── 'A'
│   ╰── 'C'
│       ╰── 'E'
├── 'B'
╰── 'D'


## Special Data Types

### Plain Strings

We can add simple string objects the same way as any other object

In [29]:
tree_str = Tree()
a = tree_str.add("A")
a.add("a1")
a.add("a2")
tree_str.add("B")
tree_str.print()

Tree<'4387679616'>
├── 'A'
│   ├── 'a1'
│   ╰── 'a2'
╰── 'B'


## Dictionaries

In [30]:
tree = Tree()
tree.add("A").up().add("B")
d = {"title": "foo", "id": 1}
tree["A"].add(d, data_id=d["id"])
tree["B"].add(d, data_id=d["id"])
tree.find(d)
# tree.print(repr="{node}")

TypeError: unhashable type: 'dict'

In [ ]:
from nutree import DictWrapper

tree = Tree()
tree.add("A").up().add("B")
d = {"title": "foo", "id": 1}
tree["A"].add(DictWrapper(d))
tree["B"].add(DictWrapper(d))
tree["B"].add(DictWrapper(d))

tree.find(d)
# tree.print(repr="{node}")

# Advanced

## Chaining

Some methods return a node instance, so we can chain calls. <br>
This allows for a more compact code and avoids some temporary variables:

In [ ]:
Tree().add("A").add("a1").up().add("a2").up(2).add("B").tree.print()